In [ ]:
#import dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Loading the Dataset
dir=f'/content/drive/MyDrive/EEG/data'
import pandas as pd
path = '/content/drive/MyDrive/Random forest SVM Decision trees/data.txt'
df = pd.read_csv(path,sep=',')
df.columns = ['0', '1', '2', '3', '4','5','6','7','8','9','10','11','12','13','target_class']
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,target_class
0,4324.62,4004.62,4293.85,4148.72,4342.05,4586.67,4097.44,4638.97,4210.77,4226.67,4207.69,4279.49,4632.82,4384.10,0
1,4327.69,4006.67,4295.38,4156.41,4336.92,4583.59,4096.92,4630.26,4207.69,4222.05,4206.67,4282.05,4628.72,4389.23,0
2,4328.72,4011.79,4296.41,4155.90,4343.59,4582.56,4097.44,4630.77,4217.44,4235.38,4210.77,4287.69,4632.31,4396.41,0
3,4326.15,4011.79,4292.31,4151.28,4347.69,4586.67,4095.90,4627.69,4210.77,4244.10,4212.82,4288.21,4632.82,4398.46,0
4,4321.03,4004.62,4284.10,4153.33,4345.64,4587.18,4093.33,4616.92,4202.56,4232.82,4209.74,4281.03,4628.21,4389.74,0


In [ ]:
from sklearn.model_selection import train_test_split
X=df[df.columns[0:14]].values # Features
y = df['target_class'] # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # 90% training and 10% test
print('X_train',X_train.shape)
print('X_test',X_test.shape)
print('y_train',y_train.shape)
print('y_test',y_test.shape)

X_train (13481, 14)
X_test (1498, 14)
y_train (13481,)
y_test (1498,)


In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
batch_size = 256
num_classes = 2
epochs = 100

In [ ]:
import keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import Convolution1D, ZeroPadding1D, MaxPooling1D, BatchNormalization, Activation, Dropout, Flatten, Dense

In [ ]:
model = Sequential()
model.add(Convolution1D(128, kernel_size=3,padding = 'same' ,activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2)))
model.add(Convolution1D(128,kernel_size=3,padding = 'same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=(2)))
model.add(Flatten())
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='sigmoid'))


In [ ]:
for i, layer in enumerate(model.layers):
       layer._name = 'layer_' + str(i)
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_0 (Conv1D)             (None, 14, 128)           512       
_________________________________________________________________
layer_1 (BatchNormalization) (None, 14, 128)           512       
_________________________________________________________________
layer_2 (MaxPooling1D)       (None, 7, 128)            0         
_________________________________________________________________
layer_3 (Conv1D)             (None, 7, 128)            49280     
_________________________________________________________________
layer_4 (BatchNormalization) (None, 7, 128)            512       
_________________________________________________________________
layer_5 (MaxPooling1D)       (None, 3, 128)            0         
_________________________________________________________________
layer_6 (Flatten)            (None, 384)              

In [ ]:
intermediate_layer_model = keras.Model(inputs=model.input,
                                       outputs=model.get_layer('layer_11').output)
X_train = np.array(X_train)
intermediate_output = intermediate_layer_model(X_train)
print(intermediate_output)
print(intermediate_output.shape)

tf.Tensor(
[[0.68633485 0.58174586 0.64819145 ... 0.38075694 0.36830875 0.2652369 ]
 [0.68633175 0.58193654 0.6481373  ... 0.3810042  0.36835182 0.26537055]
 [0.6863795  0.5818995  0.64811456 ... 0.38097632 0.36839974 0.26538855]
 ...
 [0.68611395 0.5819288  0.6482265  ... 0.38102406 0.36827537 0.26541978]
 [0.6866405  0.58181375 0.6482034  ... 0.380522   0.3679971  0.26461017]
 [0.686268   0.5817271  0.64821357 ... 0.38076007 0.36830974 0.26527986]], shape=(13481, 16), dtype=float32)
(13481, 16)


In [ ]:
intermediate_output_X_test = intermediate_layer_model(X_test)
print(intermediate_output_test.shape)

(1498, 16)
(1498, 14)


In [ ]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(intermediate_output, y_train)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y_pred = svclassifier.predict(intermediate_output_test)
print(y_pred.shape)

(1498,)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[820   0]
 [678   0]]
              precision    recall  f1-score   support

           0       0.55      1.00      0.71       820
           1       0.00      0.00      0.00       678

    accuracy                           0.55      1498
   macro avg       0.27      0.50      0.35      1498
weighted avg       0.30      0.55      0.39      1498

0.5473965287049399


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
